<a href="https://colab.research.google.com/github/potuu/BreastCancerPredictionFromMammography/blob/main/BreastCancerFromMammography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
kaggle datasets download -d farhanmonsur/breast-cancer-split-dataset

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [3]:
!kaggle datasets download -d farhanmonsur/breast-cancer-split-dataset

Dataset URL: https://www.kaggle.com/datasets/farhanmonsur/breast-cancer-split-dataset
License(s): unknown
 99% 1.54G/1.55G [00:22<00:00, 94.1MB/s]
100% 1.55G/1.55G [00:22<00:00, 75.3MB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/breast-cancer-split-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [10]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [12]:
extract_dir='/content/All Data Split'
train_dir=os.path.join(extract_dir,'/content/All Data Split/Train')
test_dir=os.path.join(extract_dir,'/content/All Data Split/Train')

In [25]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

**eğitim verilerinin yüklenmesi**

In [26]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 212421 images belonging to 2 classes.


**test verilerinin yüklenmesi**

In [27]:
test_generator=test_datagen.flow_from_directory(
    test_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 212421 images belonging to 2 classes.


**modelin oluşturulması**

In [29]:
#keras
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [30]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

**Modelin eğitilmesi** **4 SAAT 20 DAKİKA SÜRDÜ**

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

Epoch 1/10
1115/6638 ━━━━━━━━━━━━━━━━━━━━ 2:19:03 2s/step - accuracy: 0.7850 - loss: 0.5132

**Eğitim ve doğrulama doğruluğu grafiği**

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Eğitim ve Doğrulama Doğruluğu')
plt.legend()
plt.show()

**Eğitim ve doğrulama kaybı grafiği**

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Eğitim ve Doğrulama Doğruluğu')
plt.legend()
plt.show()

**Modelin tahmin yapması ve doğruluğunun hesaplanması**

In [ ]:
Y_pred = model.predict(test_generator)
y_pred = np.where(Y_pred > 0.5, 1, 0)

**Confusion Matrix Report**

In [ ]:
print('Confussion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

**Classification Report**

In [ ]:
print('Classification Report')
target_names = ['Benign', 'Malignant']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

**Doğruluk oranı (accuracy) hesaplama**

In [ ]:
accuracy=accuracy_score(test_generator.classes, y_pred)
print(f"Modelin doğruluk oranı: ½{accuracy *100:.2f}")

**görüntülerin bölünmesi ve analiz edilmesi**

In [ ]:
import random
from keras.preprocessing.image import load_img,img_to_array

tahmin sırasında görüntülerin bölünmesi ve analiz edilmesi

In [ ]:
random img_path=os.path.join(test_dir,'1', random.choice(os.path.join(test_dir,'1')))



def predict_image_with_augmentation(image_path):
    img=load_img(image_path,target_size=(150,150))
    img=img_to_array(img) / 255.0
    img=np.expand_dims(img, axis=0)

    img_gen=test_datagen.flow(img,batch_size=1)

    for i in range(5):
      aug_img=next(img_gen)[0]
      plt.imshow(aug_img)
      plt.axis('off')
      plt.show()

      pred=model.predict(np.expand_dims(aug_img,axis=0))
      if pred>=0.5:
        print(f"Model Tahmini: Kanserli(Confidence: {pred[0][0]:.2f})")
      else:
        print(f"Model Tahmini: Kansersiz(Confidence: {1-pred[0][0]:.2f})")


In [ ]:
predict_image_with_augmentation(random_img_path)